<a href="https://www.kaggle.com/code/william2020/deep-fnn-for-intent-classification?scriptVersionId=187706217" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Deep FNN for Intent Classification: Training, Validating, and Analyzing Misclassifications

### This notebook demonstrates how to train, evaluate, and analyze a neural network model for intent classification using PyTorch and sklearn.

In [26]:
!pip install -q torch scikit-learn numpy

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from kaggle_secrets import UserSecretsClient
import numpy as np
import os
import json

# Grab the intent data from datasets

In [28]:
# Load Kaggle dataset path
kaggle_data_path = "/kaggle/input/intent-data"

# Import intent data from the Kaggle dataset
import sys
sys.path.append(kaggle_data_path)
from intents import intent_data

# What's do we have?

In [29]:
num_intents = len(intent_data)
num_examples = sum(len(examples) for examples in intent_data.values())

print(f"Number of intents: {num_intents}")
print(f"Total number of examples: {num_examples}")

Number of intents: 35
Total number of examples: 1910


# Define the Neural Network Model
## The neural network model for intent classification consists of three fully connected layers with ReLU activations.

In [ ]:
class IntentClassifierModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(IntentClassifierModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 32)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(32, 16)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(16, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.fc5(out)
        return out

# Load Data
## We load the intent data, vectorize the texts using TfidfVectorizer, and encode the labels.

In [ ]:
def load_data(intent_data):
    vectorizer = TfidfVectorizer()
    texts = []
    labels = []
    original_phrases = []
    for intent, phrases in intent_data.items():
        for phrase in phrases:
            texts.append(phrase)
            labels.append(intent)
            original_phrases.append(phrase)
    X_tfidf = vectorizer.fit_transform(texts).toarray()
    label_to_index = {label: idx for idx, label in enumerate(intent_data.keys())}
    y = np.array([label_to_index[label] for label in labels])
    return X_tfidf, y, vectorizer, label_to_index, original_phrases

# Train and Save the Model
## We define a function to train the model and save its state dictionary.

In [ ]:
def train_and_save_model(intent_data, model_path):
    try:
        # Load data
        X, y, vectorizer, label_to_index, _ = load_data(intent_data)
        input_dim = X.shape[1]  # Number of input features
        output_dim = len(label_to_index)  # Number of output classes
        
        # Initialize the model
        model = IntentClassifierModel(input_dim, output_dim)
        criterion = nn.CrossEntropyLoss()  # Loss function for classification
        optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001
        
        # Convert data to PyTorch tensors
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.long)
        
        # Train the model
        num_epochs = 300  # Number of training epochs
        for epoch in range(num_epochs):
            model.train()  # Set the model to training mode
            optimizer.zero_grad()  # Zero the gradients
            outputs = model(X_tensor)  # Forward pass
            loss = criterion(outputs, y_tensor)  # Compute the loss
            loss.backward()  # Backward pass (compute gradients)
            optimizer.step()  # Update the weights
            
            # Calculate training accuracy
            _, predicted_train = torch.max(outputs.data, 1)  # Get the index of the max log-probability
            train_accuracy = accuracy_score(y_tensor, predicted_train)  # Calculate accuracy
            
            # Print the loss and accuracy every 10 epochs
            if (epoch + 1) % 10 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy:.4f}')
        
        print('Training completed')
        # Save the state dictionary of the model
        torch.save(model.state_dict(), model_path)
        print(f'Model state dictionary saved to {model_path}')
        return model, vectorizer, label_to_index
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None, None

# Validate the Model
## We define a function to validate the model, log misclassified examples, and save them to a JSON file.

In [ ]:
def validate_model(model, X_test, y_test, label_to_index, vectorizer, original_phrases, output_json_path):
    model.eval()
    misclassified_examples = []
    with torch.no_grad():
        X_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_tensor = torch.tensor(y_test, dtype=torch.long)
        outputs = model(X_tensor)
        _, predicted = torch.max(outputs.data, 1)
        accuracy = accuracy_score(y_tensor, predicted)
        
        # Log confusion matrix
        cm = confusion_matrix(y_tensor, predicted)
        print(f'Confusion Matrix:\n{cm}')
        
        # Log classification report
        labels = list(label_to_index.keys())
        report = classification_report(y_tensor, predicted, target_names=labels, labels=np.arange(len(labels)), zero_division=0)
        print(f'Classification Report:\n{report}')
        
        # Record misclassified examples
        misclassified_indices = np.where(predicted != y_tensor)[0]
        for idx in misclassified_indices:
            misclassified_example = {
                'text': original_phrases[idx],
                'true_label': labels[y_test[idx]],
                'predicted_label': labels[predicted[idx]]
            }
            misclassified_examples.append(misclassified_example)
            print(f'Misclassified Example - Text: {original_phrases[idx]}, True Label: {labels[y_test[idx]]}, Predicted Label: {labels[predicted[idx]]}')

        # Save misclassified examples to JSON file
        with open(output_json_path, 'w') as f:
            json.dump(misclassified_examples, f, indent=4)
        
        return accuracy, predicted

# Main execution

Along with exection of the training we will create a function called, plot_confusion_matrix.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Main code block
MODEL_PATH = "/kaggle/working/intent_classifier_nn.pth"  # Update this to a file path
OUTPUT_JSON_PATH = "/kaggle/working/misclassified_examples.json"  # Path to save the misclassified examples

if not MODEL_PATH:
    print(f"Model path is invalid: {MODEL_PATH}")
    exit(1)

# Train and save the model
model, vectorizer, label_to_index = train_and_save_model(intent_data, MODEL_PATH)

if model is None:
    print("Model training failed.")
    exit(1)

# Load data for validation
X, y, _, _, original_phrases = load_data(intent_data)

# Validate the model
accuracy, predicted = validate_model(model, X, y, label_to_index, vectorizer, original_phrases, OUTPUT_JSON_PATH)
print(f"Validation Accuracy: {accuracy:.4f}")

## Plot the Confusion Matrix

In [ ]:
labels = list(label_to_index.keys())
plot_confusion_matrix(y, predicted.numpy(), labels)

## Misclassified exapmles

Now that the misclassifed examples from the validation test are situated nicely in teh misclassified_examples.json, we can grab that json file in the kaggle/working directory, place the items in a df and display them nicely.

In [ ]:
import pandas as pd

OUTPUT_JSON_PATH = "/kaggle/working/misclassified_examples.json"

# Load the misclassified examples from the JSON file
with open(OUTPUT_JSON_PATH, 'r') as f:
    misclassified_examples = json.load(f)

# Convert the misclassified examples to a DataFrame for better visualization
misclassified_df = pd.DataFrame(misclassified_examples)

# Display the DataFrame
pd.set_option('display.max_colwidth', None)  # To ensure full text is displayed
display(misclassified_df)